In [1]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 80)

import random

import sys
sys.path.append("../")

from src.object import *
from src.create import *

from config.formation import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
df_name = pd.read_csv("../data/csv/NationalNames.csv")
df_name_list = list(df_name[df_name.Gender=='M'].Name.sample(500))

In [3]:
df_place = pd.read_csv("../data/csv/results.csv")
df_place = df_place.city.unique()

In [4]:
class League:
    def __init__(self, name, teams, num, category, relegation_num=0, promotion_num=0):
        self.name = name
        self.teams = teams
        self.num = num
        self.category = category
        self.team_result = {}
        self.player_result = {}
        self.champion = {}
        
        self.relegation = {}
        self.relegation_num = relegation_num
        self.promotion = {}
        self.promotion_num = promotion_num

        self.set_team_leaguename()
    
    def set_team_leaguename(self):
        for t in self.teams:
            t.league_name = self.name
    
    def set_player_result(self, competition_name, year, kind):
        for t in self.teams:
            for p in t.affilation_players:
                p.result[competition_name] = {}
                p.result[competition_name]["goal"] = 0
                p.result[competition_name]["assist"] = 0
                p.result[competition_name]["試合数"] = 0
                p.result[competition_name]["年度"] = year
                p.result[competition_name]["分類"] = kind
    
    def set_team_result(self, season_name):
        all_team_name = [s.name for s in self.teams]
        output = pd.DataFrame(np.zeros((len(all_team_name), 5)), 
                              index=all_team_name, 
                              columns=["win", "lose", "row", "得点", "失点"], 
                              dtype=np.int8)
        self.team_result[season_name] = output
    
    def cal_1year_result(self, year):
        season_name = f'{self.name}_{year}'
        
        self.team_result[season_name]["得失点差"] = self.team_result[season_name]["得点"]-self.team_result[season_name]["失点"]
        self.team_result[season_name]["Points"] = self.team_result[season_name].apply(apply_points, axis=1)
        self.team_result[season_name] = self.team_result[season_name].sort_values("Points", ascending=False)
        self.team_result[season_name]["順位"] = [f"{i}位" for i in range(1, 21)]
        self.team_result[season_name]["リーグ名"] = [f"{self.name}" for i in range(20)]
        
        for team in self.teams:
            team.result.loc[season_name] = self.team_result[season_name].loc[team.name, :]
        
        self.champion[season_name] = list(self.team_result[season_name].index)[0]
        
        # 昇格決定
        if self.category!="top":
            self.promotion[season_name] = list(self.team_result[season_name][:self.promotion_num].index)
        
        #降格決定
        if self.category!="lowest":
            self.relegation[season_name] = list(self.team_result[season_name][-self.relegation_num:].index)
        
        player_result = pd.DataFrame(columns=["名前", "ポジション", "リーグ", "年度", "チーム", "試合数", "goal", "assist", "分類"])

        for t in self.teams:
            for p in t.formation.players_flat:
                df = p.result[season_name]
                df["名前"] = p.name
                df["リーグ"] = self.name
                df["チーム"] = t.name
                df["ポジション"] = p.partification_position
                output = pd.DataFrame(df, index=[0],  
                                      columns=["名前", "ポジション", "リーグ", "年度", "チーム", "試合数", "goal", "assist", "分類"])
                player_result = pd.concat([player_result, output])
        
        player_result = player_result.reset_index(drop=True)
        self.player_result[season_name] = player_result
        
    def play_1section(self, year, sections):
        season_name = f'{self.name}_{year}'
        
        for section in sections:
            game = Game(home=self.teams[section[0]-1], 
                        away=self.teams[section[1]-1], 
                        competition_name=season_name,
                        moment_num=9,
                        random_std=15)
            game.battle()

            home_team_name = self.teams[section[0]-1].name
            away_team_name = self.teams[section[1]-1].name

            if game.result=="home":
                self.team_result[season_name].loc[home_team_name, "win"] += 1
                self.team_result[season_name].loc[away_team_name, "lose"] += 1
            elif game.result=="away":
                self.team_result[season_name].loc[away_team_name, "win"] += 1
                self.team_result[season_name].loc[home_team_name, "lose"] += 1
            else:
                self.team_result[season_name].loc[home_team_name, "row"] += 1
                self.team_result[season_name].loc[away_team_name, "row"] += 1

            self.team_result[season_name].loc[home_team_name, "得点"] += game.home_goal
            self.team_result[season_name].loc[home_team_name, "失点"] += game.away_goal

            self.team_result[season_name].loc[away_team_name, "得点"] += game.away_goal
            self.team_result[season_name].loc[away_team_name, "失点"] += game.home_goal

class Competition:
    def __init__(self, name):
        self.name = name
        self.now_round = 1
        self.section_interval = 0
        self.max_round = None
    
    def set_max_round(self, num_teams, num_section):
        for max_round in range(N):
            if num_teams < pow(2, max_round):
                break
        self.max_round = max_round
        self.section_interval = num_section//self.max_round

class ProSoccerLeague:
    def __init__(self, name, leagues):
        self.name = name
        self.leagues = leagues
        self.competition = None
        self.competition_teams = None
        self.competition_result = {}
    
    def set_competition(self, competition_name, year, num_section):
        self.competition_teams=[]

        for l in self.leagues:
            self.competition_teams.extend(l.teams)
            l.set_player_result(competition_name, year, "カップ戦")
        
        self.competition.set_max_round(len(self.competition_teams), num_section)
        random.shuffle(self.competition_teams)
        output = pd.DataFrame(columns=["チームA", "チームB", "勝利", "スコア", "ラウンド"])
        self.competition_result[competition_name] = output
    
    def play_1season(self, year, competition):
        league_calender = create_calender()
        num_section = len(league_calender)
        self.competition = competition
        self.set_competition(self.competition.name, year, num_section)
        
        for l in self.leagues:
            l.set_team_leaguename()
            season_name = f'{l.name}_{year}'
            l.set_player_result(season_name, year, "リーグ")
            l.set_team_result(season_name)
        
        for day in range(num_section):
            if day==int(self.competition.section_interval*self.competition.now_round):
                self.play_1competition_section(year)
            sections = league_calender.iloc[day, :]
            for league in self.leagues:
                league.play_1section(year, sections)
        
        for l in self.leagues:
            l.cal_1year_result(year)
        
        for index in range(len(self.leagues)):
            season_name = f"{self.leagues[index].name}_{year}"
            if self.leagues[index].category!="top":
                promotion = self.leagues[index].promotion[season_name]
                promotion_team = [t for t in self.leagues[index].teams if t.name in promotion]
                self.leagues[index].teams = [s for s in self.leagues[index].teams if s not in promotion_team]
                self.leagues[index-1].teams.extend(promotion_team)

            if self.leagues[index].category!="lowest":
                relegation = self.leagues[index].relegation[season_name]
                relegation_team = [t for t in self.leagues[index].teams if t.name in relegation]
                self.leagues[index].teams = [s for s in self.leagues[index].teams if s not in relegation_team]
                self.leagues[index+1].teams.extend(relegation_team)
    
    def play_1competition_section(self, year):
        buff_teams = self.competition_teams.copy()
        if self.competition.now_round==1:
            self.competition_teams = self.competition_teams[:(len(self.competition_teams)-pow(2, self.competition.max_round-1))*2]

        for i in range(0, len(self.competition_teams), 2):
            game_team = self.competition_teams[i:i+2]
            if len(game_team) < 2:
                continue

            cup_game = Game(home=game_team[0], 
                            away=game_team[1], 
                            competition_name=self.competition.name,
                            moment_num=9,
                            random_std=20,
                            pk=1)
            cup_game.battle()
            if cup_game.result=="home" or cup_game.result=="home-pk":
                win = game_team[0]
                lose = game_team[1]
            else:
                win = game_team[1]
                lose = game_team[0]

            if self.competition.max_round-3 >= self.competition.now_round:
                round_name = f"{self.competition.now_round}回戦"
                result_name = f"{self.competition.now_round}回戦"
            elif self.competition.max_round-2 == self.competition.now_round:
                round_name = f"準々決勝"
                result_name = f"ベスト8"
            elif self.competition.max_round-1 == self.competition.now_round:
                round_name = f"準決勝"
                result_name = f"ベスト4"
            elif self.competition.max_round == self.competition.now_round:
                round_name = f"決勝"
                result_name = f"準優勝"

            if "pk" in cup_game.result:
                score = f"{cup_game.home_goal}-{cup_game.away_goal}(pk:{cup_game.home_pk_goal}-{cup_game.away_pk_goal})"
            else:
                score = f"{cup_game.home_goal}-{cup_game.away_goal}"

            output = pd.Series([f"{game_team[0].name}({game_team[0].league_name})", 
                                f"{game_team[1].name}({game_team[1].league_name})",
                                win.name, score, round_name], 
                               index=["チームA", "チームB", "勝利", "スコア", "ラウンド"])
            self.competition_result[self.competition.name].loc[f"{game_team[0].name}-{game_team[1].name}"] = output
            lose.competition_result[self.competition.name] = result_name
            buff_teams.remove(lose)
        self.competition.now_round += 1
        self.competition_teams = buff_teams
        if len(self.competition_teams)<2:
            win.competition_result[self.competition.name] = "優勝"


In [5]:
league_team = []

for i in range(20):
    min_rate = np.random.randint(70, 78)
    max_rate = min_rate + 15
    players = random_create_players(27, 3, min_rate, max_rate, df_name_list)
    formation = random_create_formation()
    A = Team(name=df_place[i], 
             formation=formation)

    A.set_all_players(players)
    A.set_starting_players()
    A.formation.cal_team_rate()
    
    league_team.append(A)

L1 = League(name="premia", 
            teams=league_team,
            num=20,
            category='top',
            relegation_num=3)

In [6]:
league_team = []

for i in range(20):
    min_rate = np.random.randint(60, 70)
    max_rate = min_rate + 15
    players = random_create_players(27, 3, min_rate, max_rate, df_name_list)
    formation = random_create_formation()
    A = Team(name=df_place[i+20], formation=formation)

    A.set_all_players(players)
    A.set_starting_players()
    A.formation.cal_team_rate()
    
    league_team.append(A)

L2 = League(name="championship",
            teams=league_team,
            num=20,
            category='middle',
            relegation_num=3,
            promotion_num=3)

In [7]:
league_team = []

for i in range(20):
    min_rate = np.random.randint(57, 65)
    max_rate = min_rate + 15
    players = random_create_players(27, 3, min_rate, max_rate, df_name_list)
    formation = random_create_formation()
    A = Team(name=df_place[i+40], formation=formation)

    A.set_all_players(players)
    A.set_starting_players()
    A.formation.cal_team_rate()
    
    league_team.append(A)

L3 = League(name="EFL1", 
            teams=league_team,
            num=20,
            category='middle',
            relegation_num=3,
            promotion_num=3)

In [8]:
league_team = []

for i in range(20):
    min_rate = np.random.randint(55, 63)
    max_rate = min_rate + 15
    players = random_create_players(27, 3, min_rate, max_rate, df_name_list)
    formation = random_create_formation()
    A = Team(name=df_place[i+60], formation=formation)

    A.set_all_players(players)
    A.set_starting_players()
    A.formation.cal_team_rate()
    
    league_team.append(A)

L4 = League(name="EFL2", 
            teams=league_team,
            num=20,
            category='lowest',
            relegation_num=0,
            promotion_num=3)

In [9]:
year = 1990

England = ProSoccerLeague(name="england", leagues=[L1, L2, L3, L4])

for year in range(1990, 2010):
    print(year)
    competition = Competition(name=f"FA杯_{year}")

    England.play_1season(year, competition)

1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009


In [20]:
league_num = 0

year = 1997
league_name = f"{England.leagues[league_num].name}_{year}"


print([t.name for t in England.leagues[league_num].teams])

b = England.leagues[league_num].champion
pd.DataFrame(b.values(), index=b.keys(), columns=["team"])

England.leagues[league_num].team_result[league_name]

['Glasgow', 'London', 'Belfast', 'Liverpool', 'Manchester', 'Crewe', 'Stoke-on-Trent', 'Sunderland', 'Wolverhampton', 'Bangor', 'Birmingham', 'Richmond', 'Swansea', 'Paisley', 'Sheffield', 'Motherwell', 'Newark', 'Wrexham', 'Shrewsbury', 'Cardiff']


,team
premia_1990,Richmond
premia_1991,Richmond
premia_1992,Birmingham
premia_1993,Birmingham
premia_1994,Richmond
premia_1995,Birmingham
premia_1996,Birmingham
premia_1997,Richmond
premia_1998,Richmond
premia_1999,Birmingham


,win,lose,row,得点,失点,得失点差,Points,順位,リーグ名
Richmond,29,3,6,79,19,60,93,1位,premia
Birmingham,27,5,6,70,21,49,87,2位,premia
Glasgow,23,8,7,70,37,33,76,3位,premia
Crewe,22,7,9,71,34,37,75,4位,premia
Wolverhampton,20,10,8,65,47,18,68,5位,premia
Stoke-on-Trent,19,13,6,65,50,15,63,6位,premia
Swansea,17,11,10,56,39,17,61,7位,premia
Manchester,15,10,13,57,43,14,58,8位,premia
Newark,16,14,8,51,51,0,56,9位,premia
Sunderland,16,16,6,53,52,1,54,10位,premia


In [21]:
team_num = 11

print(England.leagues[league_num].teams[team_num].name)
b = England.leagues[league_num].teams[team_num].competition_result
pd.DataFrame(b.values(), index=b.keys(), columns=["ラウンド"])
England.leagues[league_num].teams[team_num].result

Richmond


,ラウンド
FA杯_1990,ベスト4
FA杯_1991,優勝
FA杯_1992,優勝
FA杯_1993,準優勝
FA杯_1994,ベスト8
FA杯_1995,優勝
FA杯_1996,2回戦
FA杯_1997,4回戦
FA杯_1998,1回戦
FA杯_1999,優勝


,win,lose,row,得点,失点,得失点差,Points,順位,リーグ名
premia_1990,32,1,5,85,15,70,101,1位,premia
premia_1991,31,3,4,86,18,68,97,1位,premia
premia_1992,29,5,4,78,17,61,91,2位,premia
premia_1993,29,5,4,75,14,61,91,2位,premia
premia_1994,31,0,7,85,11,74,100,1位,premia
premia_1995,28,4,6,82,18,64,90,2位,premia
premia_1996,27,3,8,84,24,60,89,2位,premia
premia_1997,29,3,6,79,19,60,93,1位,premia
premia_1998,29,2,7,86,19,67,94,1位,premia
premia_1999,29,3,6,78,18,60,93,2位,premia


In [22]:
England.leagues[league_num].player_result[league_name].sort_values("goal", ascending=False).head(20)
England.leagues[league_num].team_result[league_name]

,名前,ポジション,リーグ,年度,チーム,試合数,goal,assist,分類
1,Jabori,ST,premia,1997,Glasgow,38,31,5,リーグ
131,Maxmilian,ST,premia,1997,Richmond,38,30,3,リーグ
101,Gorden,ST,premia,1997,Wolverhampton,38,24,0,リーグ
121,Reid,ST,premia,1997,Birmingham,38,23,5,リーグ
51,Eligah,ST,premia,1997,Manchester,38,23,4,リーグ
81,Jermon,ST,premia,1997,Stoke-on-Trent,38,22,3,リーグ
20,Kymani,ST,premia,1997,Belfast,38,22,4,リーグ
10,Dominick,ST,premia,1997,London,38,20,2,リーグ
180,Dawit,ST,premia,1997,Shrewsbury,38,20,2,リーグ
140,Kennie,ST,premia,1997,Swansea,38,19,1,リーグ


,win,lose,row,得点,失点,得失点差,Points,順位,リーグ名
Richmond,29,3,6,79,19,60,93,1位,premia
Birmingham,27,5,6,70,21,49,87,2位,premia
Glasgow,23,8,7,70,37,33,76,3位,premia
Crewe,22,7,9,71,34,37,75,4位,premia
Wolverhampton,20,10,8,65,47,18,68,5位,premia
Stoke-on-Trent,19,13,6,65,50,15,63,6位,premia
Swansea,17,11,10,56,39,17,61,7位,premia
Manchester,15,10,13,57,43,14,58,8位,premia
Newark,16,14,8,51,51,0,56,9位,premia
Sunderland,16,16,6,53,52,1,54,10位,premia


In [23]:
for p in England.leagues[league_num].teams[team_num].affilation_players:
    if p.partification_position=="ST":
        break
print(p.name, p.main_position, p.partification_position)
df = pd.DataFrame(p.result.values(), index=p.result.keys())

df_league = df[df["分類"]=="リーグ"]
df_league.sort_values("年度")

Maxmilian ST ST


,goal,assist,試合数,年度,分類,名前,リーグ,チーム,ポジション
premia_1990,16,4,38,1990,リーグ,Maxmilian,premia,Richmond,ST
premia_1991,26,5,38,1991,リーグ,Maxmilian,premia,Richmond,ST
premia_1992,23,1,38,1992,リーグ,Maxmilian,premia,Richmond,ST
premia_1993,14,5,38,1993,リーグ,Maxmilian,premia,Richmond,ST
premia_1994,21,1,38,1994,リーグ,Maxmilian,premia,Richmond,ST
premia_1995,19,1,38,1995,リーグ,Maxmilian,premia,Richmond,ST
premia_1996,26,3,38,1996,リーグ,Maxmilian,premia,Richmond,ST
premia_1997,30,3,38,1997,リーグ,Maxmilian,premia,Richmond,ST
premia_1998,26,1,38,1998,リーグ,Maxmilian,premia,Richmond,ST
premia_1999,22,4,38,1999,リーグ,Maxmilian,premia,Richmond,ST


In [24]:
England.leagues[league_num].teams[team_num].formation.print_formation()

Dusty   Rate: 83 ( LW )
-------------------------
  PAC  70   DRI  99
  SHO  69   DEF  59
  PAS  69   PHY  59

Maxmilian   Rate: 80 ( ST )
-------------------------
  PAC  85   DRI  60
  SHO  80   DEF  60
  PAS  71   PHY  56

Mikhel   Rate: 82 ( RW )
-------------------------
  PAC  99   DRI  75
  SHO  67   DEF  47
  PAS  49   PHY  61

Heron   Rate: 85 ( CM )
-------------------------
  PAC  81   DRI  73
  SHO  70   DEF  76
  PAS  88   PHY  75

Lamarr   Rate: 82 ( CM )
-------------------------
  PAC  91   DRI  94
  SHO  48   DEF  62
  PAS  82   PHY  57

Antoinne   Rate: 81 ( CDM )
-------------------------
  PAC  83   DRI  53
  SHO  56   DEF  79
  PAS  86   PHY  79

Lorinza   Rate: 83 ( LB )
-------------------------
  PAC  82   DRI  88
  SHO  70   DEF  80
  PAS  58   PHY  80

Less   Rate: 86 ( CB )
-------------------------
  PAC  58   DRI  69
  SHO  51   DEF  85
  PAS  48   PHY  87

Jermarcus   Rate: 86 ( CB )
-------------------------
  PAC  82   DRI  56
  SHO  46   DEF  82
  PAS  

In [19]:
England.competition_result["FA杯_1990"]

,チームA,チームB,勝利,スコア,ラウンド
Cardiff-Gentilly,Cardiff(championship),Gentilly(EFL1),Cardiff,3-2,1回戦
Brussels-Portsmouth,Brussels(championship),Portsmouth(championship),Portsmouth,3-3(pk:5-6),1回戦
Dundee-St. Gallen,Dundee(championship),St. Gallen(EFL2),St. Gallen,3-4,1回戦
La Chaux-de-Fonds-Luxembourg,La Chaux-de-Fonds(EFL2),Luxembourg(EFL2),Luxembourg,0-3,1回戦
Sheffield-Dublin,Sheffield(premia),Dublin(championship),Sheffield,3-0,1回戦
Antwerp-Liège,Antwerp(EFL1),Liège(EFL2),Liège,0-0(pk:19-20),1回戦
Stoke-on-Trent-Kilmarnock,Stoke-on-Trent(premia),Kilmarnock(championship),Stoke-on-Trent,7-2,1回戦
Greenock-Shrewsbury,Greenock(championship),Shrewsbury(premia),Shrewsbury,2-5,1回戦
Aberdeen-Viña del Mar,Aberdeen(championship),Viña del Mar(EFL1),Aberdeen,8-1,1回戦
Richmond-Southampton,Richmond(premia),Southampton(championship),Richmond,7-1,1回戦


In [16]:
90//7

12

In [17]:
38//6

6